In [4]:
import os
import json
from tqdm import tqdm

import numpy as np
import torch

def tile2d(a, w=None):
    a = np.asarray(a)
    if w is None:
        w = int(np.ceil(np.sqrt(len(a))))
    th, tw = a.shape[1:3]
    pad = (w - len(a)) % w
    a = np.pad(a, [(0, pad)] + [(0, 0)] * (a.ndim - 1), 'constant')
    h = len(a) // w
    a = a.reshape([h, w] + list(a.shape[1:]))
    a = np.rollaxis(a, 2, 1).reshape([th * h, tw * w] + list(a.shape[4:]))
    return a


In [10]:
@torch.no_grad()
def torch_model_to_np(nca_model, has_bias=False):
    params = list(nca_model.parameters())

    layers = []

    i = 0
    layer1_weight = params[i][:, :, 0, 0].detach().cpu().numpy()
    i += 1
    layer1_bias = params[i][:, None].detach().cpu().numpy()
    i += 1
    layer1_params = np.concatenate([layer1_weight, layer1_bias], axis=1).T
    layer1_params = layer1_params[None, ...]
    # layer_params[:, -1] is for bias
    # layer_params[:, -3:-1] is for the positional encoding

    layers.append(layer1_params)

    if has_bias:
        layer2_weight = params[i][:, :, 0, 0].detach().cpu().numpy()
        i += 1
        layer2_bias = params[i][:, None].detach().cpu().numpy()
        i += 1
        layer2_params = np.concatenate([layer2_weight, layer2_bias], axis=1).T
        layer2_params = layer2_params[None, ...]
    else:
        layer2_weight = params[i][:, :, 0, 0].detach().cpu().numpy().T
        i += 1
        layer2_params = layer2_weight[None, ...]

    layers.append(layer2_params)

    return layers


@torch.no_grad()
def torch_model_list_to_np(model_paths, has_bias=True):
    model = torch.load(model_paths[0]).eval().cpu()
    np_params = torch_model_to_np(model, has_bias=has_bias)
    for model_path in model_paths[1:]:
        model = torch.load(model_path).eval().cpu()
        params = torch_model_to_np(model, has_bias=has_bias)
        for i, p in enumerate(params):
            np_params[i] = np.concatenate([np_params[i], p], axis=0)

    return np_params


def export_np_models_to_json(np_models, metadata):
    '''Exoprt numpy models in a form that ca.js can read.'''
    models_js = {'model_names': metadata['model_names'], 'layers': []}
    for i, layer in enumerate(np_models):
        shape = layer[0].shape
        layer = np.array(layer)  # shape: [n, c_in, fc_dim]
        if i == 0:
            c = 1
            if metadata['pos_emb']:
                c += 2
            # Replaced with np equiv. for time being so this works internally.
            # layer[:,:-1] = rearrange(layer[:,:-1], 'n (h c) w -> n (c h) w', c=fixed_filter_n)
            s = layer[:, :-c].shape
            # layer[:, :-c] = (layer[:, :-c]
            #                  .reshape(s[0], -1, fixed_filter_n, s[2])
            #                  .transpose(0, 2, 1, 3) # [n, 4, c_in, fc_dim]
            #                  .reshape(s))
        # layer = rearrange(layer, 'n h (w c) -> h (n w) c', c=4)
        # N.B. this 4 is not the fixed filter number, but a webgl implementation detail.
        # Pad when number of channels is not a multiple of 4.
        s = layer.shape
        layer = np.pad(layer, ((0, 0), (0, 0), (0, (4 - s[2]) % 4)), mode='constant')
        layer = layer.reshape(s[0], s[1], -1, 4)  # [n, 4xc_in, fc_dim // 4, 4]
        n, ht, wt = layer.shape[:3]
        w = 1
        while w < n and w * wt < (n + w - 1) // w * ht:
            w += 1
        layer = tile2d(layer, w)
        layout = (w, (n + w - 1) // w)

        scale = float(layer.max() - layer.min())
        center = float(-layer.min() / scale)
        layer = layer - layer.min()
        layer = layer / scale
        layer_flatten = layer.flatten()

        layer = np.round(layer * 255.0)
        layer = np.uint8(layer.clip(0, 255))

        layer_js = {
            'scale': scale,
            'center': center,
            'data_flatten': list(map(float, list(layer_flatten))),
            'data_shape': layer.shape,
            'shape': shape,
            'layout': layout,
            'pos_emb': (i == 0) and metadata['pos_emb'],
            'bias': True,

        }
        models_js['layers'].append(layer_js)
    return models_js


In [14]:
# Converting vector field motion models

image_path = "data/VectorFieldMotion/Appearance/"
img_names = sorted(os.listdir(image_path))
img_names = [s for s in img_names if ".jpg" in s or ".png" in s]
print(f"Number of Images: {len(img_names)}")
models_path = "out/VectorFieldMotionS/"
    
motion_names = [
    '0', '270', 'grad_0_0', 'grad_0_90',
    'circular', 'diverge', 'converge', 'hyperbolic',
    '2block_x', '2block_y', '3block', '4block',
]

# img_names = ['flames.png']
for img_name in tqdm(img_names):
    img_name = img_name.split(".")[0]
    model_pth_paths = [] 
    for motion_name in motion_names:
        model_pth_paths.append(f"{models_path}/{img_name}/{motion_name}/model.pth")
        
    np_params = torch_model_list_to_np(model_pth_paths, True)
    js_models = export_np_models_to_json(np_params, {'model_names': motion_names, 'pos_emb': True})
    json.dump(js_models, open(f"out/json_models/VectorFieldMotionS/{img_name}.json", 'w'))

Number of Images: 45


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 45/45 [00:06<00:00,  6.93it/s]
